# Clone repos and change dir

In [ ]:
# clone and install tf_unet 
! git clone https://github.com/jakeret/tf_unet.git
! cd tf_unet && python setup.py install --user

In [ ]:
# clone/update the GraduationProject
# if project does not exit, then clone the remote repo to local
# if do exit now, then pull from the remote, both the main project and submodules
import os
os.chdir("/content")

!if [ ! -d "/content/GraduationProject" ]; then \
    cd /content && \
    echo -e "\n❗️Project does not exist, pulling from remote..." && \
    git clone https://github.com/WenjayDu/GraduationProject.git && \
    cd GraduationProject && \
    git submodule update --init --recursive && \
    echo "❗️Done pulling"; \
else \
    echo -e "\n❗️Start updating..." && \
    cd /content/GraduationProject && \
    echo "🚩Now in `pwd`" && \
    git pull && \
    cd module_minc_keras && \
    echo "🚩Now in `pwd`" && \
    git checkout master && \
    git pull && \
    cd ../module_pocketflow && \
    echo "🚩Now in `pwd`" && \
    git checkout master && \
    git pull && \
    echo "❗️Done updating"; \
fi


# Prepare dataset

In [ ]:
os.chdir("/content/GraduationProject/datasets")
# decompress output.tar.bz2 and make mri dataset
!if [ ! -d "./mri" ]; then \
    tar -jxvf /content/GraduationProject/module_minc_keras/data/output.tar.bz2 -C ./ &> /dev/null && \
    mv output mri; \
fi

# download sorteo.tar.bz2 and make sorteo dataset
!if [ ! -d "./sorteo" ]; then \
    wget https://amnesia.cbrain.mcgill.ca/deeplearning/sorteo.tar.bz2 --no-check-certificate && \
    mkdir sorteo && \
    tar -jxvf sorteo.tar.bz2 -C sorteo &> /dev/null; \
fi

# prepare datasets
os.chdir("/content/GraduationProject")
from data_processing.prepare_datasets import prepare_dataset
# generate small datasets to avoid memory exceeding causing process killed on cloab
prepare_dataset(dataset_name="mri",ratios=[0.3, 0.15]) 
prepare_dataset(dataset_name="sorteo",ratios=[0.3, 0.15])

# convert generated .npy files to .tfrecords files
from data_processing.convert_npy_to_tfrecords import convert_whole_dataset
convert_whole_dataset("mri")
convert_whole_dataset("sorteo")


# Construct unet with Keras and train it

## on MRI dataset

### architecture=original, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/keras_impl_original.py \
    --dataset_name="mri" \
    --epoch_num=10

### architecture=original with BN in the contracting path, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/keras_impl_original_with_BN.py \
    --dataset_name="mri" \
    --epoch_num=10

### architecture=smaller, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/keras_impl_smaller.py \
    --dataset_name="mri" \
    --epoch_num=10

### architecture=smaller with BN in the contracting path, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/keras_impl_smaller_with_BN.py \
    --dataset_name="mri" \
    --epoch_num=10

## on SORTEO dataset

### architecture=original, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/keras_impl_original.py \
    --dataset_name="sorteo" \
    --epoch_num=10

### architecture=original with BN in the contracting path, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/keras_impl_original_with_BN.py \
    --dataset_name="sorteo" \
    --epoch_num=10

### architecture=smaller, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/keras_impl_smaller.py \
    --dataset_name="sorteo" \
    --epoch_num=10

### architecture=smaller with BN in the contracting path, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/keras_impl_smaller_with_BN.py \
    --dataset_name="sorteo" \
    --epoch_num=10

## make prediction with trained model

In [ ]:
from module_minc_keras.predict import predict
os.chdir("/content/GraduationProject")
!if [ ! -d "output/keras_impl_original_with_BN/predictions" ]; then \
    mkdir output/keras_impl_original_with_BN/predictions; \
fi

predict(model_fn='output/keras_impl_original_with_BN/saved_models/unet_model_on_mri.hdf5', 
        predict_dir='output/keras_impl_original_with_BN/predictions',
        data_dir='datasets/mri_pad_4_results/data', 
        images_fn='datasets/mri_pad_4_results/report/images.csv', 
        loss='categorical_crossentropy',
        images_to_predict='all', 
        category="test", 
        verbose=True)

# Construct unet with TensorFlow and train it

## on MRI dataset

### architecture=original, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/tf_impl_original.py \
    --train_batch_size=64 \
    --validate_batch_size=64 \
    --test_batch_size=64 \
    --dataset_name="mri" \
    --epoch_num=10

### architecture=original with BN in the contracting path, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/tf_impl_original_with_BN.py \
    --train_batch_size=64 \
    --validate_batch_size=64 \
    --test_batch_size=64 \
    --dataset_name="mri" \
    --epoch_num=10

## on SORTEO dataset

### architecture=original, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/tf_impl_original.py \
    --train_batch_size=64 \
    --validate_batch_size=64 \
    --test_batch_size=64 \
    --dataset_name="sorteo" \
    --epoch_num=10

### architecture=original with BN in the contracting path, epoch=10

In [ ]:
!python3 /content/GraduationProject/unet_constructing/tf_impl_original_with_BN.py \
    --train_batch_size=64 \
    --validate_batch_size=64 \
    --test_batch_size=64 \
    --dataset_name="sorteo" \
    --epoch_num=10


# Use ngrok to map tensorboard to a public URL to enable visit, if you want, just run it

In [ ]:
# run tensorboard
LOG_DIR="" # you must give the path of log dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


In [ ]:
# download and unzip ngrok
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip


In [ ]:
# run ngrok
get_ipython().system_raw('./ngrok http 6006 &')


In [ ]:
# map to public URL
! curl -s http://localhost:4040/api/tunnels | \
    python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])" | \
    xargs echo "the public url of tensorboard is -> "


# Freeze the graph to .pb file if you want

In [ ]:
from data_processing.freeze_and_restore_graph import freeze_graph
freeze_graph()


# Model compressing

In [ ]:
!if [ ! -d "/content/GraduationProject/module_pocketflow/compressing_with_PF" ]; then \ 
    cp -r /content/GraduationProject/compressing_with_PF /content/GraduationProject/module_pocketflow/; \
fi 

os.chdir("/content/GraduationProject/module_pocketflow")
!./scripts/run_local.sh compressing_with_PF/unet_on_mri_run.py \
        --learner=full-prec \
        --cycle_length=1 \
        --nb_threads=1 \
        --buffer_size=1 \
        --prefetch_size=1



# Make record of GPU state

In [ ]:
! while [ 1 ]; \
do echo -e `nvidia-smi`"\n\n" >> /content/gpu-record.md && \
   sleep 5; \
done